In [1]:
#importing the required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Position_Salaries.csv')

In [3]:
df.head()

,Position,Level,Salary
0,Business Analyst,1,45000
1,Junior Consultant,2,50000
2,Senior Consultant,3,60000
3,Manager,4,80000
4,Country Manager,5,110000


### Separating the independent and dependent features
#### Dependent feature


In [4]:
y = np.asarray(df['Salary'].values.tolist()) 


### Independent Feature

In [5]:
X = np.asarray(df['Level'].values.tolist())

In [6]:
X.shape

(10,)

In [7]:
y.shape

(10,)

In [8]:
# Reshaping the independent feature
X = X.reshape(-1,1)

In [ ]:
# Changing the shape from (10,) to (10,1)

In [9]:
#Reshaping the Dependent features
y = y.reshape(len(y),1) 

In [10]:
print("The shape of the independent fatures are ",X.shape)
print("The shape of the dependent fatures are ",y.shape)

The shape of the independent fatures are  (10, 1)
The shape of the dependent fatures are  (10, 1)


### The method "poly_features" concatenates polynomials of independent feature to X
### This is similar to PolynomialFeatures class from sklearn.preprocessing

In [ ]:
def poly_features(features, X):
  data = pd.DataFrame(np.zeros((X.shape[0],features)))
  for i in range(1,features+1):
    data.iloc[:,i-1] = (X**i).reshape(-1,1)
  X_poly = np.array(data.values.tolist())
  return X_poly

### The method "split_data" splits the given dataset into trainset and testset
### This is similar to the method "train_test_split" from "sklearn.model_selection"

In [ ]:
def split_data(X,y,test_size=0.2,random_state=0):
     np.random.seed(random_state)                  #set the seed for reproducible results
     indices= np.random.permutation(len(X))       #shuffling the indices
     data_test_size = int(X.shape[0] * test_size)  #Get the test size

        
    #Separating the Independent and Dependent features into the Train and Test Set
     train_indices = indices[data_test_size:]
     test_indices = indices[:data_test_size]
     X_train = X[train_indices]
     y_train = y[train_indices]
     X_test = X[test_indices]
     y_test = y[test_indices]
     return X_train, y_train, X_test, y_test

###  The method pred_to_plot returns predictions on given values and helps in
### better visualization

In [ ]:
def pred_to_plot(W_trained, X):
  prediction_values = list()
  for i in range(X.shape[0]):
    value = regressor.predict(W_trained,X[i])
    prediction_values.append(value)
  return prediction_values

### Coding the polynomialRegression Class

### Parameters:



X (array) : Independent Features
y (array) : Dependent Features/ Target Variable
W (array) : Weights 

Returns:
loss (float) : Calculated Sqaured Error Loss for y and y_pred
y_pred (array) : Predicted Target Variable

In [1]:
class polynomialRegression():

  def __init__(self):
    #No instance Variables required
    pass

  def forward(self,X,y,W):
    y_pred = sum(W * X)
    loss = ((y_pred-y)**2)/2    #Loss = Squared Error, we introduce 1/2 for ease in the calculation
    return loss, y_pred


### UpdateWeights
### Parameters:



X (array) : Independent Features
y_pred (array) : Predicted Target Variable
y_true (array) : Dependent Features/ Target Variable
W (array) : Weights
alpha (float) : learning rate
index (int) : Index to fetch the corresponding values of W, X and y 

Returns:
W (array) : Update Values of Weight